In [6]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import cv2 as cv
# Изменять размер картинок до загрузки
from skimage.transform import resize
import skimage.io as io

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from keras.layers import Input, Dense, Flatten, SimpleRNN, LSTM, GRU, Conv2D, MaxPool2D
from keras.models import Sequential

from keras import backend
from keras import metrics
from keras import preprocessing
from keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [33]:
data_dir = os.path.join(os.pardir, os.pardir,'data')

x = []       
y = []
shape = (1920,1080)#решейп это кто

for fullpath, dirs, files in os.walk(data_dir):
    if len(dirs) == 0:
        dirname = fullpath.split(os.path.sep)[-1]
        if dirname == 'open_img':
            for file in files:
                image = io.imread(os.path.join(fullpath,file))
                image = resize(image, shape)
                x.append(image)
        else:
            for file in files:
                image = io.imread(os.path.join(fullpath, file))
                image = resize(image, shape)
                y.append(image)

x = np.array(x)
y = np.array(y)